In [ ]:
# Sel 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Mengatur agar plot matplotlib tampil inline di notebook
%matplotlib inline
# Mengatur style plot seaborn
sns.set_style('whitegrid')

# Sel 2: Markdown
# 1. Memuat Data

Memuat dataset `heart_failure_clinical_records.csv` dari folder `data/`.

In [ ]:
# Sel 3: Memuat Data
# Path ke dataset. '../' berarti 'naik satu level' dari folder 'notebooks/' ke 'data/'
file_path = '../data/heart_failure_clinical_records.csv'
df = pd.read_csv(file_path)

# Tampilkan 5 baris pertama untuk verifikasi
print("Data berhasil dimuat. 5 baris pertama:")
display(df.head())

# Sel 4: Markdown
# 2. Exploratory Data Analysis (EDA)

Melakukan analisis awal untuk memahami data, sesuai dengan rencana di proposal (Bab 3.1).

In [ ]:
# Sel 5: EDA - Verifikasi Data
print("Informasi Dataset (Tipe Data & Nilai Non-Null):")
df.info()

print("\nRingkasan Statistik Fitur Numerik:")
display(df.describe())

# Sel 6: Markdown
### 2.1. Analisis Variabel Target (DEATH_EVENT)

Menganalisis distribusi kelas target untuk melihat ketidakseimbangan (imbalance) data.

In [ ]:
# Sel 7: EDA - Analisis Target
print("Distribusi Kelas Target (DEATH_EVENT):")
print(df['DEATH_EVENT'].value_counts())

# Visualisasi
plt.figure(figsize=(6, 4))
sns.countplot(x='DEATH_EVENT', data=df)
plt.title('Distribusi Kelas Target (0=Selamat, 1=Meninggal)')
plt.show()

print("\nObservasi: Data tidak seimbang (imbalanced), sesuai proposal.")

# Sel 8: Markdown
### 2.2. Analisis Fitur

Melihat distribusi dari fitur-fitur numerik dan kategorikal.

In [ ]:
# Sel 9: EDA - Distribusi Fitur Numerik
# Kita pisahkan fitur biner/kategorikal agar histogram lebih bermakna
binary_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']
numeric_features = [col for col in df.columns if col not in binary_features + ['DEATH_EVENT']]

# Plot histogram untuk fitur numerik
df[numeric_features].hist(bins=20, figsize=(15, 10))
plt.suptitle('Distribusi Fitur Numerik', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Sel 10: EDA - Distribusi Fitur Biner
# Plot countplot untuk fitur biner
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(binary_features):
    sns.countplot(x=col, data=df, hue='DEATH_EVENT', ax=axes[i])
    axes[i].set_title(f'Distribusi Fitur {col}')

# Menghapus subplot ekstra jika ada
if len(binary_features) < len(axes):
    for j in range(len(binary_features), len(axes)):
        fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

# Sel 11: Markdown
### 2.3. Analisis Korelasi (Heatmap)

Membuat heatmap korelasi untuk melihat hubungan antar semua fitur. Ini adalah salah satu visualisasi yang direncanakan di proposal (Bab 3.4).

In [ ]:
# Sel 12: EDA - Heatmap Korelasi
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Heatmap Korelasi Antar Fitur')
plt.show()

# Sel 13: Markdown
# 3. Preprocessing Data

Mempersiapkan data untuk modeling:
1.  Memisahkan fitur (X) dan target (y).
2.  Melakukan Train-Test Split (proporsi 80-20 dan *stratified*).
3.  Melakukan Feature Scaling (StandardScaler) pada data training dan testing.

In [ ]:
# Sel 14: Preprocessing - Memisahkan X dan y
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

print("Bentuk X (fitur):", X.shape)
print("Bentuk y (target):", y.shape)

In [ ]:
# Sel 15: Preprocessing - Train-Test Split
# Melakukan split 80% training dan 20% testing
# stratify=y sangat PENTING untuk memastikan proporsi kelas target tetap sama
# di data train dan test, karena data kita imbalanced.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,  # random_state untuk hasil yang konsisten
    stratify=y
)

print("Bentuk X_train:", X_train.shape)
print("Bentuk X_test:", X_test.shape)
print("\nDistribusi target di y_train:")
print(y_train.value_counts(normalize=True))
print("\nDistribusi target di y_test:")
print(y_test.value_counts(normalize=True))

In [ ]:
# Sel 16: Preprocessing - Feature Scaling
# Logistic Regression sensitif terhadap skala fitur.
# Kita wajib melakukan scaling.

# 1. Buat scaler
scaler = StandardScaler()

# 2. Fit scaler HANYA pada data training (X_train)
scaler.fit(X_train)

# 3. Transformasi X_train dan X_test
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ubah kembali menjadi DataFrame agar mudah dibaca dan disimpan
# (Penting: gunakan 'columns=X.columns' untuk menjaga nama kolom)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

print("Data training setelah scaling (5 baris pertama):")
display(X_train_scaled_df.head())

# Sel 17: Markdown
# 4. Menyimpan Data yang Telah Diproses

Menyimpan data `X_train_scaled`, `X_test_scaled`, `y_train`, dan `y_test` ke folder `data/` agar bisa langsung digunakan oleh Notebook `02_Modeling_Logistic_Regression.ipynb`.

In [ ]:
# Sel 18: Menyimpan Data
# Tentukan path penyimpanan
path_prefix = '../data/'

# Simpan data yang sudah bersih dan terpisah
X_train_scaled_df.to_csv(path_prefix + 'X_train_scaled.csv', index=False)
X_test_scaled_df.to_csv(path_prefix + 'X_test_scaled.csv', index=False)
y_train.to_csv(path_prefix + 'y_train.csv', index=False)
y_test.to_csv(path_prefix + 'y_test.csv', index=False)

print("Data telah diproses dan disimpan di folder 'data/':")
print(f"- X_train_scaled.csv ({X_train_scaled_df.shape})")
print(f"- X_test_scaled.csv ({X_test_scaled_df.shape})")
print(f"- y_train.csv ({y_train.shape})")
print(f"- y_test.csv ({y_test.shape})")

# Sel 19: Markdown
# Selesai

EDA dan Preprocessing telah selesai. Data bersih siap untuk modeling di notebook `02_Modeling_Logistic_Regression.ipynb`.